In [ ]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

In [ ]:
import ipywidgets as ipw
#from ipywidgets import AppLayout, Button, Layout
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

import panel as pn
import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer
hv.extension('bokeh')

import numpy as np
import time


Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Oscillating reaction - Lotka Volterra model

The Lotka-Volterra model is the simplest model that shows an oscillatory behaviour in the concentration of the intermediate species.
It can be used to model chemical reactions in constant flow reactors or other non chemical systems, such as oscillations in the population of preys and predators in an ecosystem.
In this model there are three competing reactions:
\begin{equation*}
\mathrm{A} + \mathrm{M} \xrightarrow{k_1} 2\mathrm{A} \\
\mathrm{A} + \mathrm{B} \xrightarrow{k_2} 2\mathrm{B} \\
\mathrm{B} \xrightarrow{k_3} \mathrm{P}
\end{equation*}
where M is the feedstock of reactant(s) that is assumed to be constant, A and B are intermediate species that are in the reactor to make the reaction happen and P represents the product(s) that is removed from the reactor.
The first two reactions are auto-catalytic because from one intermediate molecules two are produced, while the last reaction consumes the intermediate B to form the product.

\begin{equation*}
v_1 = k_1\mathrm{[A][M]} \\
v_2 = k_2\mathrm{[A][B]} \\
v_3 = k_3\mathrm{[B]}
\end{equation*}

Hence, the rate of change of A and B and the rate of formation of the products are

\begin{equation*}
\frac{\mathrm{d[A]}}{\mathrm{d}t} = +k_1\mathrm{[A][M]} - k_2\mathrm{[A][B]} \\
\frac{\mathrm{d[B]}}{\mathrm{d}t} = +k_1\mathrm{[A][B]} - k_3\mathrm{[B]} \\
\frac{\mathrm{d[P]}}{\mathrm{d}t} = +k_3\mathrm{[B]}.
\end{equation*}

Because this model represent a constant flow reaction, the concentration of M is unchanged and counld be included in the $k_1$ rate constant. Analogously, because the producs are removed they don't play a role in the kinetics of the system. In order to numerically solve this problem, we make the assumption that the rates can be considered as constant in small time intervals, $\mathrm{d}t$, and solve the above problem iteratively, where we update the rates only when a time $\mathrm{d}t$ has passed.
At every cycle, the change in concentration of the intermediates can therefore be written as

\begin{equation*}
\mathrm{d[A]} = \mathrm{d}t\ (k_1\mathrm{[A][M]} - k_2\mathrm{[A][B]}) \\
\mathrm{d[B]} = \mathrm{d}t\ (k_1\mathrm{[A][B]} - k_3\mathrm{[B]})
\end{equation*}

Thus, we can compute the changes in the concentrations of the intermediates between time $t$ and time $t+\mathrm{d}t$ as

\begin{equation*}
\quad \mathrm{[A]}(t+\mathrm{d}t) = \mathrm{[A]}(t) + \mathrm{d}t\ (k_1\mathrm{[A][M]} - k_2\mathrm{[A][B]}) \\
\mathrm{[B]}(t+\mathrm{d}t) = \mathrm{[B]}(t) + \mathrm{d}t\ (k_1\mathrm{[A][B]} - k_3\mathrm{[B]})
\end{equation*}

So, by interating the above procedure for thousands of cycles we can predic the time evolution of the concentrations of A and B over time.

The default parameters in the boxes below should show the time evolution of the concentrations of A and B over a couple of oscillations (first graph). The second graph will instead show the concentration of B vs the concentration of A.

Try altering some of the parameters and see how the result changes.
In particular, verify that an excessive increase in the length of the time interval between successive updates of the rates, $\mathrm{d}t$, will break the numeric solution of the problem.


In [ ]:
#initialize Pipe object
pipe1 = Pipe(data=[])
pipe2 = Pipe(data=[])

#Create a line chart
plot1 = hv.DynamicMap(hv.Curve, streams=[pipe1])
plot2 = hv.DynamicMap(hv.Curve, streams=[pipe2])

plot1.opts(xlabel='t',ylabel='[A] or [B]')

#Adjust plot size
(plot1*plot2).opts(opts.Curve(width=500, show_grid=True, framewise=True))

# plot2.opts(xlabel='',ylabel='')



In [ ]:
pipe3 = Pipe(data=[])
plot3 = hv.DynamicMap(hv.Curve, streams=[pipe3])
(plot3).opts(opts.Curve(width=500, show_grid=True, framewise=True))
plot3.opts(xlabel='[A]',ylabel='[B]')


In [ ]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

def measure(dt,kx,ky,kz,cm,ca,cb):
    dx = kx * dt * cm * ca
    dy = ky * dt * ca * cb
    dz = kz * dt * cb
    return dx,dy,dz
    
def calc(btn):
    out_P.clear_output()

    x,y,z = [],[],[]
    
    kx = float(k1.value)
    ky = float(k2.value)
    kz = float(k3.value)
    cm = float(ConcM.value)
    ca = float(ConcA.value)
    cb = float(ConcB.value)
    cp = 0
    dt = float(timeStep.value)
    tmax = 100*dt
    
    elapT = 0.
    res = pd.DataFrame(columns=["Time [min]" , "[A]", "[B]", "[P]"])

    # Measurement result
    ns = int(nSteps.value)
    for istep in range(0,ns):
        for j in range(0,100):
            dd = measure(dt,kx,ky,kz,cm,ca,cb)
            ca = ca + dd[0] - dd[1]
            cb = cb + dd[1] - dd[2]
            cp = cp + dd[2]
            elapT = elapT + dt

        if (elapT > tmax):
            tmax = tmax + 1000*dt

        x.extend([elapT])
        y.extend([ca])
        z.extend([cb])
        pipe1.send({'x':np.array(x),'y':np.array(y)})
        pipe2.send({'x':np.array(x),'y':np.array(z)})
        pipe3.send({'x':np.array(y),'y':np.array(z)})

        var_list = []
        var_list.append(elapT)            
        var_list.append(ca)
        var_list.append(cb)
        var_list.append(cp)
        res.loc[len(res)] = var_list
    
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(respath):
        os.remove(respath)

    x,y,z = [],[],[]
    pipe1.send({'x':np.array(x),'y':np.array(y)})
    pipe2.send({'x':np.array(x),'y':np.array(z)})
    pipe3.send({'x':np.array(y),'y':np.array(z)})
    
    with out_P:
        out_P.clear_output()

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# ---
reset(btn_reset)

# --- create the boxes and sliders
rows = []

label_layout = ipw.Layout(width='200px')

k1 = ipw.Text("0.1")
k2 = ipw.Text("0.1")
k3 = ipw.Text("0.05")
box01 = ipw.Box([ipw.Label('k$_1$  :  ',layout=label_layout),k1])
box02 = ipw.Box([ipw.Label('k$_2$  :  ',layout=label_layout),k2])
box03 = ipw.Box([ipw.Label('k$_3$  :  ',layout=label_layout),k3])
rows.append(ipw.HBox([box01, box02, box03]))

ConcA = ipw.Text("1")
ConcB = ipw.Text("1")
ConcM = ipw.Text("1")
box11 = ipw.Box([ipw.Label('[A]$_0$  :  ',layout=label_layout),ConcA])
box12 = ipw.Box([ipw.Label('[B]$_0$  :  ',layout=label_layout),ConcB])
box13 = ipw.Box([ipw.Label('[M]$_0$  :  ',layout=label_layout),ConcM])
rows.append(ipw.HBox([box11, box12, box13]))

nSteps = ipw.Text("100")
timeStep = ipw.Text("0.002")
box21 = ipw.Box([ipw.Label('Number of Steps  :  ',layout=label_layout),nSteps])
box22 = ipw.Box([ipw.Label('Timestep  :  ',layout=label_layout),timeStep])
rows.append(ipw.HBox([box21, box22]))



rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)